<a href="https://colab.research.google.com/github/MrcCap/AnotherTest4PACS/blob/main/Mask_VDenseLayer%26Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing stuff

In [ ]:
import torch

from torch.profiler import profile, ProfilerActivity, record_function
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
from torch import optim
from torch.nn.parameter import Parameter
from torch.autograd import Variable
from torchsummary import summary
import torch.nn.functional as F
from torch import Tensor
from torch.nn import functional as F, init
from torch.nn.parameter import Parameter

import networkx as nx
import matplotlib.pyplot as plt


In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import numpy as np
from tqdm import tqdm

from scipy.stats import norm

import math

import torch.nn as nn
from torch.autograd import Variable
import time
from tqdm import tqdm

In [ ]:
# defining transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert images to tensor
    transforms.Normalize((0.5,), (0.5,)), # Normalize the images
    transforms.Lambda(lambda x: torch.flatten(x))
])

In [ ]:
# Step 3: Load the dataset
train_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
batch_size = 2048


loaders = {
    'train' : torch.utils.data.DataLoader(train_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1),

    'test'  : torch.utils.data.DataLoader(test_data,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          num_workers=1),
}
loaders

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:04<00:00, 2116509.70it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 558697.59it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 3392018.43it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 6833044.75it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



{'train': <torch.utils.data.dataloader.DataLoader at 0x7b52ddb00c10>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x7b52ddb01360>}

# Layer definition

In [ ]:
def single_linear(x, weights, bias):
    return torch.nn.functional.linear(x, weights, bias)

def single_sample(noise, std, mean):
    return torch.nn.functional.linear(noise, std, mean)

def parallel_linear(x, weight_realizations, bias):
    if x.dim() == 2:
        return torch.vmap(single_linear, in_dims = (None, 0, None))(x, weight_realizations, bias)
    return torch.vmap(single_linear, in_dims = (0, 0, None))(x, weight_realizations, bias)

def parallel_sample(noise, std, means):
    return torch.vmap(single_sample, in_dims = (None, None, 1), out_dims=1)(noise, std, means)

In [ ]:
class VariationalDenseLayer(nn.Module):
    def __init__(self, input_shape, output_shape, mask=None, multivariate=False, alpha=0.05) -> None:
        super(VariationalDenseLayer, self).__init__()

        self.input_shape = input_shape
        self.output_shape = output_shape
        self.multivariate = multivariate
        self.alpha = alpha

        self.in_features = torch.prod(torch.tensor(input_shape)).item()
        self.out_features = torch.prod(torch.tensor(output_shape)).item()

        self.set_weights()


    def set_weights(self, mask=None, mean_w_values=None, std_w_values=None):

        if mask is None:
            self.mask = torch.ones(self.out_features, self.in_features)
            active_mean_w = None
            active_std_w = None
        else:
            self.mask = mask
            active_mean_w = torch.masked_select(self.mean_w, self.mask)
            active_std_w = torch.masked_select(self.std_w, self.mask)

        self.active_weights = int(self.mask.sum().item())
        self.mean_w = Parameter(torch.empty(self.active_weights), requires_grad=True)
        self.std_w = Parameter(torch.empty(self.active_weights), requires_grad=True)

        if active_mean_w is None and mean_w_values is None:
            self.mean_w.data.fill_(0.)
        elif active_mean_w is None:
            self.mean_w.data.fill_(mean_w_values)
        else:
            self.mean_w.data.copy_(active_mean_w)

        if active_std_w is None and std_w_values is None:
            self.std_w.data.fill_(0.)
        elif active_std_w is None:
            self.std_w.data.fill_(std_w_values)
        else:
            self.std_w.data.copy_(active_std_w)

In [ ]:
class VariationalDenseLayer(nn.Module):
    def __init__(self, input_shape, output_shape, mask = None, multivariate = False, alpha = 0.05) -> None:
        super(VariationalDenseLayer, self).__init__()

        self.input_shape = input_shape
        self.output_shape = output_shape
        self.multivariate = multivariate
        self.alpha = alpha

        self.in_features = torch.prod(torch.tensor(input_shape)).item()
        self.out_features = torch.prod(torch.tensor(output_shape)).item()

        self.set_weights()

        self.mask_cholesky = torch.ones((self.out_features, int(self.in_features*(self.in_features-1)/2)))
        self.cholesky_idx = torch.nonzero(self.mask_cholesky, as_tuple=False)

        self.active_cholesky = int(self.mask_cholesky.sum().item())
        self.chol_w = Parameter(torch.empty(self.active_cholesky), requires_grad=self.multivariate)
        self.idx_tril = torch.tril_indices(self.in_features, self.in_features,-1)

        self.mean_b = Parameter(torch.empty(self.out_features), requires_grad=True)

        self.mean_b.data.fill_(0.)
        self.chol_w.data.fill_(0.001*self.multivariate)


    def set_weights(self, mask=None, mean_w_values=None, std_w_values=None):
        if mask is None:
            self.mask = torch.ones(self.out_features, self.in_features, dtype=torch.bool)
            active_mean_w = None
            active_std_w = None
        else:
            self.mask = mask
            active_mean_w = torch.masked_select(self.get_mean(), self.mask)
            active_std_w = torch.masked_select(self.get_std(), self.mask)

        self.mask_idx = torch.nonzero(self.mask, as_tuple=False)
        self.active_weights = int(self.mask.sum().item())
        self.mean_w = Parameter(torch.empty(self.active_weights), requires_grad=True)
        self.std_w = Parameter(torch.empty(self.active_weights), requires_grad=True)

        if active_mean_w is None and mean_w_values is None:
            self.mean_w.data.fill_(0.)
        elif active_mean_w is None:
            self.mean_w.data.fill_(mean_w_values)
        else:
            self.mean_w.data.copy_(active_mean_w)

        if active_std_w is None and std_w_values is None:
            self.std_w.data.fill_(0.01)
        elif active_std_w is None:
            self.std_w.data.fill_(std_w_values)
        else:
            self.std_w.data.copy_(active_std_w)

    def get_mean(self) -> torch.tensor:
        weight_tensor = torch.zeros(self.out_features, self.in_features)
        weight_tensor[self.mask_idx[:,0], self.mask_idx[:,1]] = self.mean_w
        return weight_tensor

    def get_std(self) -> torch.tensor:
        std_tensor = torch.zeros(self.out_features, self.in_features)
        std_tensor[self.mask_idx[:,0], self.mask_idx[:,1]] = self.std_w
        return std_tensor

    def get_chol(self)-> torch.tensor:
        cholesky_tril = torch.zeros(self.out_features, self.in_features, self.in_features)
        cholesky_tril[:, range(self.in_features),range(self.in_features)] = self.get_std()
        cholesky_tril[:, self.idx_tril[0], self.idx_tril[1]] = self.chol_w.view(self.out_features, int(self.in_features*(self.in_features-1)/2))
        return cholesky_tril

    def significant_weights(self) -> torch.Tensor:
        z_score = torch.abs(self.get_mean()) / torch.sqrt(self.get_std() ** 2)
        return (z_score > norm.ppf(1 - self.alpha/2))

    def update_mask(self):
        self.set_weights(mask=self.significant_weights())

    def sample_weights(self) -> torch.Tensor:
        if self.multivariate:
            return torch.einsum("ij, oij -> oi", torch.randn(self.in_features, self.in_features), self.get_chol()) + self.get_mean()
        return torch.einsum("i, oi -> oi", torch.randn(self.in_features), self.get_std()) + self.get_mean()

    def forward(self, x) -> torch.Tensor:
        return torch.nn.functional.linear(x, self.sample_weights(), self.mean_b)

In [ ]:
def single_linear(x, weights, bias):
    return torch.nn.functional.linear(x, weights, bias)

def parallel_linear(x, weight_realizations, bias):
    if x.dim() == 2:
        return torch.vmap(single_linear, in_dims = (None, 0, None))(x, weight_realizations, bias)
    return torch.vmap(single_linear, in_dims = (0, 0, None))(x, weight_realizations, bias)

In [ ]:
class VariationalNet(nn.Module):
    def __init__(self, layer_sizes, layer_masks = None) -> None:
        super(VariationalNet, self).__init__()
        # Store the actual shapes
        self.layer_sizes = layer_sizes
        self.layer_masks = layer_masks if layer_masks is not None else [None] * (len(layer_sizes) - 1)

        # Initialize layers with corresponding masks
        self.layers = nn.ModuleList([
            VariationalDenseLayer(in_size, out_size, mask, multivariate=False)
            for (in_size, out_size), mask in zip(zip(self.layer_sizes[:-1], self.layer_sizes[1:]), self.layer_masks)
        ])

    def realize_layers(self, num_samples):
        def sample_all_layers(sample_index):
            return [layer.sample_weights() for layer in self.layers]
        # Use vmap to vectorize over samples
        return torch.vmap(sample_all_layers, randomness='different')(torch.arange(num_samples))

    def update_masks(self):
        for layer in self.layers:
            layer.update_mask()

    def forward(self, x, num_samples):
        # First, realize the weights
        realized_weights = self.realize_layers(num_samples)
        for layer, realized_layer in zip(self.layers, realized_weights):
            x = parallel_linear(x, realized_layer, layer.mean_b)
            if layer != self.layers[-1]:
                x = torch.nn.LeakyReLU(negative_slope=0.1)(x)

        return x


In [ ]:
def plot_weights_and_covariance(means, chol, in_features, out_features):
    # Reshape weights_mu into 28x28 images
    img_size = int(np.sqrt(in_features))
    covariance_matrices = torch.bmm(chol, chol.transpose(1, 2))

    # Plotting
    fig, axes = plt.subplots(out_features, 2, figsize=(10, out_features * 5))

    for i in range(out_features):
        # Plot weights_mu
        axes[i, 0].imshow(means[i].view(img_size, img_size).cpu().detach().numpy(), cmap='viridis')
        axes[i, 0].set_title(f'Weights (Output Neuron {i+1})')
        axes[i, 0].axis('off')

        # Plot covariance matrix
        axes[i, 1].imshow(covariance_matrices[i].cpu().detach().numpy(), cmap='viridis')
        axes[i, 1].set_title(f'Covariance Matrix (Output Neuron {i+1})')
        axes[i, 1].axis('off')

    plt.tight_layout()
    plt.show()

In [104]:
def plot_weights(means):
    # Reshape weights_mu into 28x28 images
    img_size = int(np.sqrt(means.shape[0]))
    out_features = means.shape[1]

    # Plotting
    fig, axes = plt.subplots(out_features, figsize=(10, out_features * 5))

    for i in range(out_features):
        # Plot weights_mu
        axes[i].imshow(means[:,i].view(img_size, img_size).cpu().detach().numpy(), cmap='viridis')
        axes[i].set_title(f'Weights (Output Neuron {i+1})')
        axes[i].axis('off')

    plt.tight_layout()
    plt.show()

In [50]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

def parallel_loss(logits, target):
    def single_loss(logit):
        return torch.nn.functional.cross_entropy(logit, target, reduction='none')
    return torch.vmap(single_loss, in_dims=0)(logits)

def train(model, train_loader, val_loader, num_epochs, device, num_samples=10, update_masks = False, lr = 0.001):
    for epoch_idx in range(num_epochs):
        print(f"Epoch {epoch_idx+1}/{num_epochs}")
        print("Total trainable params:" + str(sum(p.numel() for p in model.parameters() if p.requires_grad)))

        # Training
        optimizer = optim.Adam(model.parameters(), lr=lr)
        model.train()
        train_loss = 0.0
        train_correct = 0
        total_train_samples = 0

        train_loader_tqdm = tqdm(train_loader, desc="Training", leave=True)


        for batch_idx, (data, target) in enumerate(train_loader_tqdm):
            data, target = data.to(device), target.to(device)
            batch_size = data.size(0)

            optimizer.zero_grad()

            # Forward pass with multiple weight realizations
            output = model(data, num_samples)  # Shape: [num_samples, batch_size, num_classes]

            # Compute parallel loss
            losses = parallel_loss(output, target)  # Shape: [num_samples, batch_size]

            # Average loss over samples and batch
            loss = losses.mean()

            loss.backward()
            optimizer.step()

            train_loss += loss.item() * batch_size

            # Compute accuracy using the mean prediction over samples
            mean_output = output.mean(dim=0)
            pred = mean_output.argmax(dim=1, keepdim=True)
            train_correct += pred.eq(target.view_as(pred)).sum().item()
            total_train_samples += batch_size

            # Calculate batch accuracy and average loss
            batch_acc = 100. * train_correct / total_train_samples
            batch_loss = train_loss / total_train_samples

            # Update tqdm progress bar with current loss and accuracy
            train_loader_tqdm.set_postfix({'Loss': f'{batch_loss:.4f}', 'Acc': f'{batch_acc:.2f}%'})

        train_loss /= total_train_samples
        train_acc = 100. * train_correct / total_train_samples
        if update_masks:
            print("Updating masks...")
            model.update_masks()

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

In [51]:
train_loader = loaders['train']
model = VariationalNet(layer_sizes=[784, 128, 64, 32, 10])

print('TRAINING + MASKING')
train(
    model=model,
    train_loader=train_loader,
    val_loader=None,
    num_epochs=10,
    device=torch.device("cpu"),
    num_samples=100,
    update_masks=True,
    lr = 0.01
)

TRAINING + MASKING
Epoch 1/10
Total trainable params:222058


Training: 100%|██████████| 30/30 [01:31<00:00,  3.05s/it, Loss=2.0415, Acc=20.24%]


Updating masks...
Train Loss: 2.0415, Train Acc: 20.24%
Epoch 2/10
Total trainable params:51818


Training: 100%|██████████| 30/30 [01:31<00:00,  3.06s/it, Loss=1.1995, Acc=58.45%]


Updating masks...
Train Loss: 1.1995, Train Acc: 58.45%
Epoch 3/10
Total trainable params:35080


...

Updating masks...
Train Loss: 0.1537, Train Acc: 95.48%
Epoch 9/10
Total trainable params:19354


Training: 100%|██████████| 30/30 [01:29<00:00,  2.99s/it, Loss=0.1398, Acc=95.84%]


Updating masks...
Train Loss: 0.1398, Train Acc: 95.84%
Epoch 10/10
Total trainable params:18628


Training: 100%|██████████| 30/30 [01:31<00:00,  3.04s/it, Loss=0.1287, Acc=96.12%]

Updating masks...
Train Loss: 0.1287, Train Acc: 96.12%


In [52]:
print('TRAINING + MASKING')
train(
    model=model,
    train_loader=train_loader,
    val_loader=None,
    num_epochs=3,
    device=torch.device("cpu"),
    num_samples=200,
    update_masks=True,
    lr = 0.001
)

TRAINING + MASKING
Epoch 1/3
Total trainable params:18034


Training: 100%|██████████| 30/30 [02:58<00:00,  5.96s/it, Loss=0.1051, Acc=96.87%]


Updating masks...
Train Loss: 0.1051, Train Acc: 96.87%
Epoch 2/3
Total trainable params:18006


Training: 100%|██████████| 30/30 [02:56<00:00,  5.87s/it, Loss=0.1013, Acc=96.94%]


Updating masks...
Train Loss: 0.1013, Train Acc: 96.94%
Epoch 3/3
Total trainable params:17988


Training: 100%|██████████| 30/30 [02:57<00:00,  5.91s/it, Loss=0.0993, Acc=97.01%]


Updating masks...
Train Loss: 0.0993, Train Acc: 97.01%
